In [1]:

import torch
import torch
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), 'core'))
from zeropi import ZeroPi
import general 



In [2]:
#create ZeroPi

#paramaters to pptimise over
EJ = torch.tensor(10.00, requires_grad=True, dtype=torch.double)
EL = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
ECJ = torch.tensor(20, requires_grad=True, dtype=torch.double)
EC = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
dEJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
dCJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
ECS= torch.tensor(0.1, requires_grad=True, dtype=torch.double)

#fixed params of system
flux = 0.5
ng = 0.1

#fixed params of computation
ncut = 30
hamiltonian_creation_solution = "manual_discretization_davidson"
discretization_dim = 100


zeropi = ZeroPi(EJ=EJ, 
                 EL=EL , 
                 ECJ=ECJ, 
                 EC=EC, 
                 dEJ=dEJ,
                 ECS = ECS, 
                 dCJ=dCJ, 
                 flux = flux,
                 ng=ng, 
                 ncut = ncut,
                 discretization_dim =discretization_dim,
                 hamiltonian_creation_solution=hamiltonian_creation_solution)



In [3]:

eigvals, eigvecs = zeropi.esys()


 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 169UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 182UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 163UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 188UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clo

In [4]:
dE = eigvals[0] -eigvals[1]
dE

tensor(-0.0018, dtype=torch.float64, grad_fn=<SubBackward0>)

In [5]:
dE.backward()

In [3]:
eigvals, eigvecs = zeropi.esys()
t = general.t2_rate(zeropi, eigvecs, True)

 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 179UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 200UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 169UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/AD_superconducting_qubit/ADOSC/core/zeropi.py: 210UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clo

TypeError: 'ZeroPi' object is not subscriptable

In [19]:
import torch

# Define a utility to generate a series of numbers for simplicity
sequence = torch.arange(0, 10000)

# Create the indices and values for the 'noise' tensor
noise_indices = torch.stack([sequence, sequence])
noise_values = torch.full((10000,), -10.+0.j, dtype=torch.complex128)

# Construct the 'noise' sparse tensor
noise = torch.sparse_coo_tensor(indices=noise_indices, values=noise_values, size=(10000, 10000))

# You'll need to provide the full list of values for 'ground' for it to be exact.
# Here's a placeholder for the 'ground' tensor:
ground_values = torch.tensor([8.3409e-06+0.j, -7.3005e-06+0.j] + [0.j] * 9998, dtype=torch.complex128)

# Construct the 'ground' sparse tensor
ground = torch.sparse_coo_tensor(indices=sequence.unsqueeze(0), values=ground_values, size=(10000,))


print(ground)
ground_2d = ground.reshape(10000, 1)

print(torch.transpose(ground.to(torch.complex128),0,-1))



RuntimeError: reshape is not implemented for sparse tensors

In [ ]:
result = torch.sparse.mm(noise, ground_2d)
rate = torch.mv(noise.to(torch.complex128), torch.transpose(ground.to(torch.complex128),-1,0))
